In [ ]:
!pip install snowflake-snowpark-python[pandas]==1.5.0
# !pip install snowflake-connector-python
!pip install snowflake-ml-python==1.0.10

In [3]:
!pip install snowflake[ml]

     |████████████████████████████████| 450kB 1.5MB/s eta 0:00:01
     |████████████████████████████████| 1.8MB 28.8MB/s eta 0:00:01
     |████████████████████████████████| 122kB 56.7MB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/ec/57/56b9bcc3c9c6a792fcbaf139543cee77261f3651ca9da0c93f5c1221264b/python_dateutil-2.9.0.post0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/9e/68/b3496e1d59e8c8b302df7fa5c4ce851a687b8b8ddb92d6f6254bfce77009/snowflake_snowpark_python-1.17.0-py3-none-any.whl
     |████████████████████████████████| 409kB 58.9MB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/25/89/dce01a35d354159dcc901e3c7e7eb3fe98de5cb3639c6cd39518d8830caa/scikit_learn-1.3.2-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/96/43/dae06432d0c4b1dc9e9149ad37b4ca8384cf6eb7700cd9215b177b914f0a/cloudpickle-3.0.0-py3-none-any.whl
  Using cached https://files.python

  Using cached https://files.pythonhosted.org/packages/41/24/0b023b6537dfc9bae2c779353998e3e99ac7dfff4222fc6126650e93c3f3/filelock-3.14.0-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/79/fd/4525835d3e42e2db30169d42215ce74762f447fcbf01ed02f74f59bd74cb/cryptography-42.0.7-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/e5/3e/741d8c82801c347547f8a2a06aa57dbb1992be9e948df2ea0eda2c8b79e8/idna-3.7-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/54/a7/2104f674a5a6845b04c8ff01659becc6b8978ca410b82b94287e0b1e018b/pyOpenSSL-24.1.0-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/68/13/2aa1f0e1364feb2c9ef45302f387ac0bd81484e9c9a4c5688a322fbdfd08/platformdirs-4.2.2-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/ea/ac/e9e77bc385729035143e54cc8c4785bd480eaca9df17565963556b0b7a93/cffi-1.16.0-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_

You should consider upgrading via the 'pip install --upgrade pip' command.


In [1]:
from snowflake.snowpark import Session

In [ ]:
import configparser
def get_session():
    parser = configparser.ConfigParser()
    # Add the credential file name here
    parser.read('/notebooks/notebooks/config.ini')

    connection_params = dict(user=parser['Credentials']['user'], 
                         password=parser['Credentials']['password'], 
                         account=parser['Credentials']['account'], 
                         warehouse=parser['Credentials']['warehouse'], 
                         database=parser['Credentials']['database'],
                         schema=parser['Credentials']['schema'], 
                         role=parser['Credentials']['role'])
    print("connection_params = ",  connection_params)


    session = Session.builder.configs(connection_params).create()
    return session

In [ ]:
session = get_session()
# session.close()
session

In [ ]:
REGISTRY_DATABASE_NAME = "test_database_prak"
REGISTRY_SCHEMA_NAME = "test_schema_prak"

In [2]:
from snowflake.ml.registry import model_registry

ModuleNotFoundError: No module named 'snowflake.ml'

In [ ]:
!pip freeze | grep -i snow